In [2]:
import matplotlib.pyplot as plt
import numpy as np
import os
import xlrd
import pandas as pd
# import seaborn as sns
# from sklearn.linear_model import LinearRegression
# from sklearn.preprocessing import FunctionTransformer
# from sklearn.preprocessing import OneHotEncoder
# from sklearn.pipeline import Pipeline
# from sklearn.compose import ColumnTransformer

# from sklearn.tree import DecisionTreeRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.neighbors import KNeighborsRegressor
# from sklearn.model_selection import train_test_split
# from sklearn.metrics import mean_squared_error
# from sklearn.preprocessing import Binarizer
# from sklearn.tree import DecisionTreeClassifier
%matplotlib inline
%config InlineBackend.figure_format = 'retina'  # Higher resolution figures

# Publich Schools Data (pubshcls.csv)

In [24]:
schools = pd.read_csv('Datasets/pubschools_info.csv')

schools = schools.loc[schools['County'] == 'San Diego']

cols_to_drop = ["NCESDist","NCESSchool","StreetAbr","MailStreet","MailStrAbr","MailCity",
               "MailZip","MailState","Ext","FaxNumber","Email","Phone","WebSite","CharterNum",
                "FundingType","DOC","DOCType","EdOpsCode","EdOpsName","Magnet","FederalDFCDistrictID",
                "AdmFName","AdmLName","AdmEmail","YearRoundYN",'Charter','State',
                'City', 'District', 'OpenDate',"SOC", "SOCType", "EILName","GSoffered","GSserved",'ClosedDate']

schools = schools.drop(cols_to_drop, axis=1)

In [25]:
clean1 = schools.loc[schools['StatusType'] == 'Active'] #Keeps only active schools
clean3 = clean1.loc[clean1['School'] != 'No Data'] #drops district records - nonschools
clean3['LastUpDate'] = pd.to_datetime(clean3['LastUpDate']) #change lastUpDate to datetime obj col
clean3 = clean3.reset_index().drop('index',axis = 1)
schools = clean3

schools = schools.replace("No Data",np.nan) #changed "No Data" into NaNs
schools['Latitude'] = schools['Latitude'].astype(float) #changed latitutde to float dtype
schools['Longitude'] = schools['Longitude'].astype(float) #changed longitude to float dtype
schools = schools.drop(['StatusType','County'],axis = 1) #dropped statustype and county col since its all active and SD county

small_zip = schools['Zip'].str.slice(0,5).astype(float)
schools = schools.assign(SmallZip=small_zip)

C:\Users\Andrew Chin\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [26]:
schools

,CDSCode,School,Street,Zip,EILCode,Virtual,Latitude,Longitude,LastUpDate,SmallZip
0,37103710108548,Iftin Charter,5465 El Cajon Boulevard,92115-3620,ELEM,N,32.757908,-117.07814,2020-08-13,92115.0
1,37103710115998,San Pasqual Academy,17701 San Pasqual Valley Road,92025-5301,HS,C,33.090580,-116.95009,2019-02-13,92025.0
2,37103710120485,Davila Day,540 G Street,91910-3604,ELEM,N,32.635041,-117.08832,2019-02-13,91910.0
3,37103710120493,Monarch,1625 Newton Avenue,92113-1012,ELEMHIGH,C,32.702375,-117.15045,2020-08-24,92113.0
4,37103710128520,San Diego County Community,"6401 Linda Vista Road, Room 216",92111-7399,ELEMHIGH,C,32.769578,-117.17984,2020-08-21,92111.0
5,37103710128538,San Diego County Court,2801 Meadow Lark Drive,92123,HS,C,32.794147,-117.15396,2019-02-13,92123.0
6,37103710135202,HOPE Infant-Toddler Special Education,6401 Linda Vista Road,92111-7319,PS,N,32.769578,-117.17984,2018-03-14,92111.0
7,37103710136085,Scholarship Prep - Oceanside,4070 Mission Avenue,92057-6402,ELEM,N,33.232373,-117.31700,2020-08-27,92057.0
8,37103710137695,Community Montessori,"1441 Montiel Road, Suite 143",92026-2242,ELEM,N,33.129426,-117.11209,2020-08-13,92026.0
9,37103710137752,Dimensions Collaborative,"1441 Montiel Road, Suite 143",92026-2242,ELEMHIGH,N,33.129426,-117.11209,2020-08-13,92026.0


# Misc. Stats

In [32]:
num_schools = len(schools)
unique_small_zips = schools['SmallZip'].nunique()
unique_zips = len(schools['Zip'].unique())
total_zips = len(schools['Zip'])

In [33]:
print("There are {} unique SmallZips out of {}".format(unique_small_zips, total_zips))
print("There are {} unique Zips out of {}.".format(unique_zips,total_zips))

There are 101 unique SmallZips out of 790
There are 703 unique Zips out of 790.


In [42]:
num_preschool = len(schools[schools['EILCode'] == 'PS'])
school_types = {}
for code in schools['EILCode'].unique():
    school_types[code] = (schools[schools['EILCode'] == code])
    
school_types['A'] #wier

,CDSCode,School,Street,Zip,EILCode,Virtual,Latitude,Longitude,LastUpDate,SmallZip
166,37681063730025,Escondido Adult,220 West Crest,92025,A,C,33.131196,-117.08846,2020-07-16,92025.0
191,37681303730199,Foothills Adult Education Center,1550 Melody Lane,92019-2015,A,C,32.797130,-116.92436,2018-08-17,92019.0
192,37681303730249,El Cajon Adult Center,1075 East Madison Avenue,92021-6226,A,C,32.800160,-116.94594,2019-08-13,92021.0
193,37681303730256,Steele Canyon Adult Center,12440 Campo Road,91978-2331,A,C,32.730297,-116.92517,2018-08-17,91978.0
201,37681303732419,Grossmont Adult,1550 Melody Lane,92019,A,C,32.797130,-116.92436,2020-08-25,92019.0
301,37682963735834,Poway Adult,13626 Twin Peaks Road,92064-3034,A,C,32.978786,-117.03586,2016-12-08,92064.0
333,37683043730900,Ramona Adult,720 Ninth Street,92065-2348,A,N,33.039329,-116.86387,2020-06-26,92065.0
401,37683383730280,San Diego Adult,4777 Imperial Avenue Room 407,92113,A,F,32.703991,-117.09202,2020-05-15,92113.0
620,37684113730132,Montgomery Adult,3240 Palm Avenue,92154-1507,A,N,32.585313,-117.06205,2019-08-26,92154.0
625,37684113731106,San Ysidro Adult Education Center,4220 Otay Mesa Road,92173,A,N,32.558871,-117.04133,2019-08-26,92173.0


# Important Codes

### EILCodes
* A – Adult
* ELEM – Elementary
* ELEMHIGH – Elementary-High Combination
* HS – High School
* INTMIDJR – Intermediate/Middle/Junior High
* PS – Preschool
* UG – Ungraded

### Virtual
* F = Exclusively Virtual – The school has no physical building where students meet with each other or with teachers, all instruction is virtual.
* V = Primarily Virtual – The school focuses on a systematic program of virtual instruction but includes some physical meetings among students or with teachers.
* C = Primarily Classroom – The school offers virtual courses but virtual instruction is not the primary means of instruction.
* N = Not Virtual – The school does not offer any virtual instruction.
* P = Partial Virtual – The school offers some, but not all, instruction through virtual instruction. Note: This value was retired and replaced with the Primarily Virtual and Primarily Classroom values beginning with the 2016–17 school year.

# Summary

Got rid of the columns that would not help in our project. Changed Dates to date time objects and numbers to floats. Also split the zip into a "SmallZip" which consists of only the first 5 digits. Filtered the schools to show only the active schools (not the closed nor merged schools)